In [ ]:
import os
os.chdir('/Users/miin/univ/4/2/DSP/project/data')

In [ ]:
import pandas as pd
import glob
import re
from datetime import datetime, timedelta
import os

# 1. 원본 데이터 로딩 및 병합
# - 중고 iPhone 15 관련 CSV 파일을 모두 읽어서 하나의 DataFrame으로 통합
print("[(1/5) 데이터 로딩 중...]")

raw_patterns = ["iphone15_resume_*.csv", os.path.join('data', 'iphone15_resume_*.csv')]
source_files = []
for pattern in raw_patterns:
    source_files.extend(glob.glob(pattern))

source_files = list(dict.fromkeys(source_files))

if source_files:
    merged_pref = [f for f in source_files if 'merged' in os.path.basename(f)]
    if merged_pref:
        source_files = merged_pref

df_list = []
if source_files:
    for filename in source_files:
        try:
            temp = pd.read_csv(filename)
            df_list.append(temp)
            print(f" -> {filename} 로딩 완료 ({len(temp)} rows)")
        except Exception as e:
            print(f" -> {filename} 로딩 실패: {e}")
    if df_list:
        df = pd.concat(df_list, ignore_index=True)
    else:
        df = pd.DataFrame()
else:
    if os.path.exists("iphone15_wtime.csv"):
        df = pd.read_csv("iphone15_wtime.csv")
        print(" -> iphone15_wtime.csv 로딩 완료")
    else:
        print("CSV 파일을 찾을 수 없습니다. 빈 데이터프레임을 생성합니다.")
        df = pd.DataFrame()

if not df.empty:
    # 중복 제거 (Listing ID 기준)
    if 'listing_id' in df.columns:
        df = df.drop_duplicates(subset=['listing_id'], keep='last')

    # 판매완료 및 예약중 데이터만 사용
    if 'status' in df.columns:
        df = df[df['status'].isin(['판매완료', '예약중'])]

    print(f"-> 총 {len(df)}개의 유효 데이터 로드 완료")

# 2. 노이즈 데이터 제거 (Cleaning)
# - 매입/구매 희망, 세트 상품 등 노이즈 데이터 필터링
print("[(2/5) 스팸 및 노이즈 제거 중...]")

if not df.empty:
    spam_keywords = ["매입", "구함", "구해요", "삽니다", "교환", "케이스", "필름", "파손", "부품"]
    pattern = '|'.join(spam_keywords)

    df = df[~df['title'].str.contains(pattern, case=False, na=False)]
    
    # 세트 상품 제거 (애플워치, 에어팟 등 포함)
    # "/"로 구분된 여러 제품이 있는 경우 (예: "아이폰15 / 애플워치")
    set_keywords = ["애플워치", "에어팟", "세트", "번들", "watch", "airpod", "airpods"]
    set_pattern = '|'.join(set_keywords)
    # 제목과 설명 모두 확인
    title_has_set = df['title'].fillna('').str.contains(set_pattern, case=False, na=False)
    desc_has_set = df['description'].fillna('').str.contains(set_pattern, case=False, na=False)
    
    # "/"로 구분된 여러 제품 패턴 (예: "아이폰15 / 애플워치9 / 에어팟")
    # 단, "아이폰15/128GB" 같은 건 제외하기 위해 주변에 제품명이 있는지 확인
    has_multiple_products = df['title'].fillna('').str.contains(r'/\s*(애플워치|에어팟|watch|airpod)', case=False, na=False) | \
                           df['description'].fillna('').str.contains(r'/\s*(애플워치|에어팟|watch|airpod)', case=False, na=False)
    
    df = df[~(title_has_set | desc_has_set | has_multiple_products)]
    
    df = df[(df['price'] >= 400000) & (df['price'] <= 3000000)]

    print(f"-> 클리닝 후 남은 데이터: {len(df)}개")

# 3. 파생 변수 생성 (Feature Engineering)
# - 날짜/모델/용량/배터리/등급 파생 변수 생성
print("[(3/5) 날짜, 배터리, 등급 추출 중...]")

def parse_date(date_str):
    now = datetime.now()
    if pd.isna(date_str) or str(date_str).strip() == "":
        return now

    date_str = str(date_str).strip()

    try:
        target_date = now

        if any(x in date_str for x in ["방금", "분 전", "시간 전", "초 전"]):
            return now
        elif "일 전" in date_str:
            match = re.search(r'(\d+)', date_str)
            days = int(match.group(1)) if match else 0
            target_date = now - timedelta(days=days)
        elif "주 전" in date_str:
            match = re.search(r'(\d+)', date_str)
            weeks = int(match.group(1)) if match else 0
            target_date = now - timedelta(weeks=weeks)
        elif "달 전" in date_str or "개월 전" in date_str:
            match = re.search(r'(\d+)', date_str)
            months = int(match.group(1)) if match else 0
            target_date = now - timedelta(days=months*30)
        elif "년 전" in date_str:
            match = re.search(r'(\d+)', date_str)
            years = int(match.group(1)) if match else 0
            target_date = now - timedelta(days=years*365)
        else:
            clean_str = re.sub(r'[^0-9.\-]', '', date_str).strip('.')
            target_date = pd.to_datetime(clean_str)

        return target_date
    except Exception:
        return now

if not df.empty and 'date_raw' in df.columns:
    df['listing_date'] = df['date_raw'].apply(parse_date)
    df['date_clean'] = df['listing_date'].dt.strftime("%Y-%m-%d")

    launch_date = pd.to_datetime("2023-10-13")
    df['days_since_launch'] = (df['listing_date'] - launch_date).dt.days
    df['weeks_since_launch'] = (df['days_since_launch'] // 7) + 1

# 모델 세부 분류

def clean_model(row):
    text = (str(row.get('spec_model', '')) + " " + str(row.get('title', ''))).upper()
    if "PRO MAX" in text or "프로맥스" in text or "프맥" in text:
        return "Pro Max"
    elif "PRO" in text or "프로" in text:
        return "Pro"
    elif "PLUS" in text or "플러스" in text:
        return "Plus"
    return "Normal"

if not df.empty:
    df['final_model'] = df.apply(clean_model, axis=1)

# 용량 통일

def clean_storage(row):
    text = (str(row.get('spec_storage', '')) + " " + str(row.get('title', ''))).upper()
    if "1TB" in text or "1테라" in text:
        return 1024
    match = re.search(r'(128|256|512)', text)
    if match:
        return int(match.group(1))
    match_desc = re.search(r'(128|256|512)', str(row.get('description', '')).upper())
    if match_desc:
        return int(match_desc.group(1))
    return None

if not df.empty:
    df['final_storage'] = df.apply(clean_storage, axis=1)

# 배터리 효율 추출

def extract_battery(text):
    text = str(text)
    patterns = [
        r'배터리\s?[:의]?\s?(\d{2,3})', r'효율\s?[:의]?\s?(\d{2,3})',
        r'성능\s?[:의]?\s?(\d{2,3})', r'(\d{2,3})\s?%', r'(\d{2,3})\s?프로', r'(\d{2,3})\s?pro'
    ]
    for pat in patterns:
        match = re.search(pat, text, re.IGNORECASE)
        if match:
            bat = int(match.group(1))
            if 80 <= bat <= 100:
                return bat
    return None

if not df.empty:
    df['battery_clean'] = df['description'].apply(extract_battery)
    df['battery_clean'] = df['battery_clean'].fillna(df['title'].apply(extract_battery))

# 3-5. 상태 등급 (Grade) 추론

def clean_grade(text):
    text = str(text).upper()
    if any(k in text for k in ["미개봉", "SEALED", "포장 미개봉", "포장미개봉"]):
        return "S (Unopened)"
    elif any(k in text for k in ["S급", "신동급", "최상급", "상태 최상", "기스 없", "기스X", "기스없", "기스 1도", "새것 같", "미사용"]):
        return "A (Like New)"
    elif any(k in text for k in ["A급", "생활기스", "찍힘", "파손", "B급", "C급"]):
        return "B/C (Used)"
    else:
        return "Unknown"

if not df.empty:
    df['grade'] = df['title'].apply(clean_grade)

# 고급 휴리스틱 기반 등급 분류 로직 (Advanced Heuristic)
# - 제목/설명/배터리 정보를 활용해 S/A/B/C를 보다 세밀하게 재분류

def advanced_grade_classifier(row):
    title = str(row.get('title', '')).strip()
    desc = str(row.get('description', '')).strip()
    text = (title + " " + desc).upper()

    s_keywords = ["미개봉", "SEALED", "포장 미개봉", "포장미개봉"]
    a_grade_keywords = ["S급", "신동급", "최상급", "상태 최상", "기스 없", "기스X", "기스없", "기스 1도", "기스1도", "새것 같", "흠집없", "흠집X", "미사용"]
    negation_tokens = ["없", "無", "무", "X", "NO", "없음", "없어요", "없습니다", "없다", "무파손", "무흠집", "무잔상"]

    explicit_pat = r'(상태|등급)\s*[:\->]\s*([SABC])'
    match = re.search(explicit_pat, text)
    if match:
        grade = match.group(2)
        if grade == 'S' and any(k in text for k in s_keywords):
            return 'S'
        if grade == 'A':
            bat = row.get('battery_clean')
            has_a_signals = any(k in text for k in a_grade_keywords)
            if pd.notnull(bat) and bat == 100:
                has_a_signals = True
            if has_a_signals:
                return 'A'
        if grade in ['B', 'C']:
            return grade

    def keyword_has_issue(keyword):
        matches = list(re.finditer(keyword, text))
        if not matches:
            return False
        for m in matches:
            start = max(0, m.start() - 2)
            end = min(len(text), m.end() + 15)
            window = text[start:end]
            window_compact = window.replace(" ", "")
            if any(token in window for token in negation_tokens) or any(token in window_compact for token in negation_tokens):
                continue
            return True
        return False

    # C급: 명시적 하자 키워드
    critical_faults = ["파손", "깨짐", "금감", "금가", "멍", "잔상", "화면 줄", "화면줄", "페이스아이디 안됨", "페이스 아이디 안됨", "기능 고장", "부품용", "부품 용", "C급"]
    if any(keyword_has_issue(k) for k in critical_faults):
        return 'C'
    
    # C급: 사용감 많음, 금가 있음 등
    heavy_usage_keywords = ["사용감 많", "사용감 많음", "사용감 많아", "사용감 많습니다", "금가 있", "금가 있음", "금가 있어", "금가 있습니다"]
    if any(kw in text for kw in heavy_usage_keywords):
        return 'C'

    if '찍힘' in text:
        chip_neg = ["찍힘 없", "찍힘없", "찍힘X", "찍힘 X", "찍힘제외", "찍힘 아니"]
        if not any(n in text for n in chip_neg):
            return 'B'

    if any(k in text for k in s_keywords):
        return 'S'

    if any(k in text for k in a_grade_keywords):
        return 'A'

    if '리퍼' in text and ('미사용' in text or '받음' in text or '새' in text):
        return 'A'

    try:
        bat = row.get('battery_clean')
        if pd.notnull(bat) and bat == 100:
            return 'A'
    except Exception:
        pass

    return 'B'

if not df.empty:
    print("[(3-5) AI Logic 기반 등급 분류 수행 중...]")
    df['prompt_grade'] = df.apply(advanced_grade_classifier, axis=1)
    
    # B급 재분류: 가격과 조건을 고려하여 A/C로 재배치
    b_mask = df['prompt_grade'] == 'B'
    b_data = df[b_mask].copy()
    
    if len(b_data) > 0:
        # A급 후보: 가격이 높고 조건이 좋은 B급
        a_candidate_price_threshold = b_data['price'].quantile(0.75)  # 상위 25%
        a_candidates = b_data[
            (b_data['price'] >= a_candidate_price_threshold) &
            (
                (b_data['battery_clean'] >= 95) |  # 배터리 95% 이상
                (b_data['title'].str.contains('리퍼', case=False, na=False)) |  # 리퍼
                (b_data['title'].str.contains('풀박스', case=False, na=False))  # 풀박스
            )
        ]
        
        # C급 후보: 가격이 낮고 사용감/하자가 있는 B급
        c_candidate_price_threshold = b_data['price'].quantile(0.25)  # 하위 25%
        c_candidates = b_data[
            (b_data['price'] <= c_candidate_price_threshold) &
            (
                (b_data['description'].fillna('').str.contains('사용감', case=False, na=False)) |
                (b_data['description'].fillna('').str.contains('기스', case=False, na=False)) |
                (b_data['description'].fillna('').str.contains('흔적', case=False, na=False)) |
                (b_data['battery_clean'] < 85)  # 배터리 85% 미만
            )
        ]
        
        # 재분류 적용
        df.loc[a_candidates.index, 'prompt_grade'] = 'A'
        df.loc[c_candidates.index, 'prompt_grade'] = 'C'
        
        print(f"  -> B급에서 A급으로 재분류: {len(a_candidates)}개")
        print(f"  -> B급에서 C급으로 재분류: {len(c_candidates)}개")

# 4. 분석용 최종 데이터 저장
# - 모델/용량/배터리/등급 정보가 정리된 최종 CSV 세 가지를 생성
print("[(4/5) 최종 파일 저장 중...]")

final_columns = [
    'listing_id', 'date_clean', 'days_since_launch', 'weeks_since_launch',
    'status', 'final_model', 'final_storage',
    'battery_clean', 'prompt_grade', 'grade', 'price', 'views', 'hearts', 'title', 'url', 'description'
]

if not df.empty:
    cols_to_save = [c for c in final_columns if c in df.columns]
    df_final = df[cols_to_save]

    df_bat_exist = df_final.dropna(subset=['battery_clean'])
    df_bat_missing = df_final[df_final['battery_clean'].isnull()]

    df_final.to_csv("iphone15_cleaned_all.csv", index=False, encoding="utf-8-sig")
    df_bat_exist.to_csv("iphone15_battery_O.csv", index=False, encoding="utf-8-sig")
    df_bat_missing.to_csv("iphone15_battery_X.csv", index=False, encoding="utf-8-sig")

    print("\n" + "="*50)
    print("🎉 전처리 완료!")
    print(f"1. 전체 데이터: iphone15_cleaned_all.csv ({len(df_final)}개)")
    print(f"2. 배터리 O: iphone15_battery_O.csv ({len(df_bat_exist)}개)")
    print(f"3. 배터리 X: iphone15_battery_X.csv ({len(df_bat_missing)}개)")
    print("="*50)

# 5. 기본 통계 확인
# - 최종 등급 분포 등 간단한 데이터 요약 출력
if not df.empty:
    print("\n[등급별 분포]")
    print(df_final['prompt_grade'].value_counts())

[(1/5) 데이터 로딩 중...]
 -> iphone15_resume_final_merged.csv 로딩 완료 (8319 rows)
-> 총 6169개의 유효 데이터 로드 완료
[(2/5) 스팸 및 노이즈 제거 중...]
-> 클리닝 후 남은 데이터: 5892개
[(3/5) 날짜, 배터리, 등급 추출 중...]


/var/folders/ln/hnq8444x7_975h6bksjrlhqr0000gn/T/ipykernel_9295/586352515.py:77: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_multiple_products = df['title'].fillna('').str.contains(r'/\s*(애플워치|에어팟|watch|airpod)', case=False, na=False) | \
/var/folders/ln/hnq8444x7_975h6bksjrlhqr0000gn/T/ipykernel_9295/586352515.py:78: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['description'].fillna('').str.contains(r'/\s*(애플워치|에어팟|watch|airpod)', case=False, na=False)


[(3-5) AI Logic 기반 등급 분류 수행 중...]
  -> B급에서 A급으로 재분류: 87개
  -> B급에서 C급으로 재분류: 25개
[(4/5) 최종 파일 저장 중...]

🎉 전처리 완료!
1. 전체 데이터: iphone15_cleaned_all.csv (5892개)
2. 배터리 O: iphone15_battery_O.csv (543개)
3. 배터리 X: iphone15_battery_X.csv (5349개)

[등급별 분포]
prompt_grade
B    4848
A     950
S      57
C      37
Name: count, dtype: int64


In [47]:

df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5892 entries, 0 to 8317
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   listing_id          5892 non-null   object 
 1   date_clean          5892 non-null   object 
 2   days_since_launch   5892 non-null   int64  
 3   weeks_since_launch  5892 non-null   int64  
 4   status              5892 non-null   object 
 5   final_model         5892 non-null   object 
 6   final_storage       5445 non-null   float64
 7   battery_clean       543 non-null    float64
 8   prompt_grade        5892 non-null   object 
 9   grade               5892 non-null   object 
 10  price               5892 non-null   int64  
 11  title               5892 non-null   object 
 12  url                 5892 non-null   object 
 13  description         166 non-null    object 
dtypes: float64(2), int64(3), object(9)
memory usage: 819.5+ KB
